In [64]:

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
import logging
import time
import telebot
import threading

 
# Configurazione delogger
logging.basicConfig(level=logging.INFO)
 
API_TOKEN = '7842681987:AAE1ajtfLfGnt8R_pjbaTf3y0_SM90NC5K4'  # Inserisci il tuo token Telegram
bot = telebot.TeleBot(API_TOKEN)

chrome_driver_path = r"C:\Users\Miriam\Desktop\Selenium\chromedriver-win64\chromedriver.exe"
 

# DIRECTORY DI DESTINAZIONE DEI FILE
#PATH = r"C:\Users\Miriam\Desktop\Selenium"
 
def init_driver():
    service = Service(chrome_driver_path)
    options = Options()
    options.add_argument("--headless")  # Disabilita l'interfaccia grafica
    options.add_argument("--enable-unsafe-swiftshader")
    options.add_argument("--disable-webgl")
    driver = Chrome(service=service, options=options)
    return driver
 
 
 
time.sleep(2)
 
def accetta_cookie(driver):
    try:
        # Attendi che l'elemento sia presente e cliccabile
        WebDriverWait(driver, 10).until(
             EC.element_to_be_clickable((By.ID, "sp-cc-accept"))
        ).click()
        logging.info("Cookie accettati con successo.")
    except NoSuchElementException:
        logging.error("Il bottone per accettare i cookie non è stato trovato.")
    except ElementClickInterceptedException:
        logging.error("Il click sul bottone dei cookie è stato bloccato.")
    except Exception as e:
        logging.error(f"Errore imprevisto: {str(e)}")



def ricerca_elemento(driver):
   driver= init_driver()
   driver.get("https://www.amazon.it/")
   accetta_cookie(driver)
   try: 
       search_bar = WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.ID,"twotabsearchtextbox"))
        )
       search_bar.send_keys("iphone 15 pro")
       logging.info("Ricerca eseguita correttamente")
     
   except TimeoutException:
       logging.error("troppo tempo passato per la ricerca")
   except Exception as e:
       logging.error(f"Errore nella ricerca: {str(e)}")






       
def visualizza(driver):
    search_button = driver.find_element(By.ID, "nav-search-submit-button")
    search_button.click()
    
    try:

        input=WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.a-size-base-plus.a-color-base.a-text-normal'))
          )
         
        products = driver.find_elements(By.CSS_SELECTOR, "div.s-main-slot div.s-result-item")

    for product in products:
        title = product.find_element(By.CSS_SELECTOR, "span.a-size-medium.a-color-base.a-text-normal").text
        try:
                price = product.find_element(By.CSS_SELECTOR, "span.a-price-whole").text
                # Stampa il titolo e il prezzo del prodotto
                print(f"Titolo: {title}, Prezzo: {price}")
        except Exception:
                # Se non riesce a trovare il prezzo, ignora l'elemento
                continue


# Gestione dei comandi /start e /help
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(message, "Ciao! Inviami il nome di un prodotto da cercare su Amazon.")
 
# Gestione dei messaggi con testo
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    product_name = message.text
    bot.reply_to(message, f"Cerco '{product_name}' su Amazon, attendi...")
 
    # Esegui lo scraping su Amazon
    products = search_amazon(product_name)
 
    if products:
        for product in products:
            bot.send_message(message.chat.id, f"Prodotto: {product['title']}\nPrezzo: {product['price']} €")
    else:
        bot.send_message(message.chat.id, "Non ho trovato nessun prodotto corrispondente.")
 
# Funzione per eseguire il bot in un thread separato
def run_bot():
    bot.infinity_polling()
 
# Esegui il bot in un thread separato
threading.Thread(target=run_bot).start()

ricerca_elemento(driver)
visualizza(driver)

driver.quit()


    
 

SyntaxError: expected 'except' or 'finally' block (417082551.py, line 89)